In [1]:
from collections import Counter, defaultdict, namedtuple
from functools import partial
import pandas as pd
import numpy as np
import math, random
import glob

Current exercise has the following steps:
1. create a DECODER model that will discriminate computer code and identify the language by the code features.
2. the input training and test files needs to be prepared.
3. Target Structure to feed into decoding model:
    - training set: [file ID, [set of features, known code])
    - testing set: [file ID, [set of features, known code])
3. the discriminating features of code are not yet defined. 
4. the files need to be processed for input.
5. Need to decide which classifier is useful.

Strategy and Initial observations:
1. Envisioning a set of data that includes a row number, a profile of regex distributions, and the code type. 
2. Need to determine how to read in a file name, capture the file extension and associate it with the regex text profile.
3. Immerse self in the TextBlob software. 
4. Need to understand the role of glob.glob.


Loading test data:
1. glob the test set of files: 1 -> 32
2. append the test.csv code label (answer key for code file 1 to 32)
   - There are 2-4 code assignments per code type in the test data.
   - Target training to test ratio should between 60/40 and 75/25.
   - Therefore, need a minimum of 16 training snippets per code to cover the minimum 25% (4test/16train).
3. initial train/testing will focus on one translater.
4. quick build - a test and a benchmark. [start with ONE]
   - build a test for Python.
   - build a training set for Python

Will start by getting control of the test data:

In [31]:
def read_snippets_test(num):
    snips=[]
    for i in range(1, num):
        filetest = glob.glob('test/{}'.format(i))
        for files in filetest:
            with open(files) as f:
                snips.append(f.read())
    return snips

In [32]:
sniptest = read_snippets_test(32)

In [33]:
type(sniptest)

list

In [34]:
len(sniptest)

31

In [45]:
sniptest[0]

'(defn cf-settings\n  "Setup settings for campfire. Required information is your api-token, ssl connection\n  true or false, and your campfire sub-domain."\n  [token ssl sub-domain]\n  {:api-token token, :ssl ssl, :sub-domain sub-domain})\n\n(defn room\n  "Sets up the room to send events too. Pass in the settings created with cf-settings\n  and the room name"\n  [settings room-name]\n  (cf/room-by-name settings room-name))\n\n(defn campfire_message\n  "Formats an event into a string"\n  [e]\n  (str (join " " ["Riemann alert on" (str (:host e)) "-" (str (:service e)) "is" (upper-case (str (:state e))) "- Description:" (str (:description e))])))\n\n(defn campfire\n  "Creates an adaptor to forward events to campfire. The campfire event will\n  contain the host, state, service, metric and description.\n  Tested with:\n  (streams\n    (by [:host, :service]\n      (let [camp (campfire \\"token\\", true, \\"sub-domain\\", \\"room\\")]\n        camp)))"\n  [token ssl sub-domain room-name]\n  (

In [36]:
sniptestnames = pd.read_csv('test.csv',sep=',',header=None)

In [37]:
type(sniptestnames)

pandas.core.frame.DataFrame

In [38]:
len(sniptestnames)

32

In [50]:
list(sniptestnames[1])[:3]

['clojure', 'clojure', 'clojure']

In [40]:
len(list(sniptestnames[1]))

32

In [47]:
sniptestnames[1][0]

'clojure'

In [48]:
sniptest[0]

'(defn cf-settings\n  "Setup settings for campfire. Required information is your api-token, ssl connection\n  true or false, and your campfire sub-domain."\n  [token ssl sub-domain]\n  {:api-token token, :ssl ssl, :sub-domain sub-domain})\n\n(defn room\n  "Sets up the room to send events too. Pass in the settings created with cf-settings\n  and the room name"\n  [settings room-name]\n  (cf/room-by-name settings room-name))\n\n(defn campfire_message\n  "Formats an event into a string"\n  [e]\n  (str (join " " ["Riemann alert on" (str (:host e)) "-" (str (:service e)) "is" (upper-case (str (:state e))) "- Description:" (str (:description e))])))\n\n(defn campfire\n  "Creates an adaptor to forward events to campfire. The campfire event will\n  contain the host, state, service, metric and description.\n  Tested with:\n  (streams\n    (by [:host, :service]\n      (let [camp (campfire \\"token\\", true, \\"sub-domain\\", \\"room\\")]\n        camp)))"\n  [token ssl sub-domain room-name]\n  (

It appears the sniptest data is a list of the code from the files.  The sniptestnames data is a DataFrame.

Want to merge the name file with the code in the same order.  

First try:  convert list of code to a DataFrame and then merge the name column.